In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as Bs 
import re 
import lxml
import time 

In [2]:
def get_html(url):
    time.sleep(1)
    res = requests.get(url)
    res.raise_for_status() 
    return res.content

def get_soup(html):
    return Bs(html, "html.parser")

def get_href(soup):
    return [i if i.startswith("/") else "/" + i for i in [i["href"] for i in soup.find_all("a")[:-1]]]

def generate_url(href):
    return [f"https://planecrashinfo.com{i}" for i in href]

# int(test_soup.find("title").text)
def generate_url_outside(href):
    return [f"https://planecrashinfo.com/{int(i.split('-')[0][1:])}{i}" for i in href]

def decompress_list(lis):
    return [m for i in lis for m in i]

def extract_data(soup_table):
    accident_data = {}
    # Loop through rows and extract key-value pairs
    for row in soup_table[1:]:  # Skip header rows
        key_cell = row.find('td', align='right')
        value_cell = row.find('td', align='left')
        if key_cell and value_cell:
            key = key_cell.text.strip().replace(':', '')  # Remove colon and trailing spaces
            value = value_cell.text.strip()
            accident_data[key] = value
    return accident_data

def first_page_pass(url):
    html = get_html(url)
    soup = get_soup(html)
    href = get_href(soup)
    href_url = generate_url(href)
    return href_url

def second_page_pass(href_url):
    return [generate_url_outside(i) for i in [get_href(i) for i in [get_soup(i) for i in [get_html(i) for i in href_url]]]]

def third_final_pass(good_url):
    return [extract_data(i) for i in [get_soup(j).find_all("tr") for j in [get_html(i) for i in good_url ]]]

def save_csv(my_list,name):
    j = pd.DataFrame(my_list)
    j.to_csv(f"data/{name}", index=False)

In [3]:
url = "https://www.planecrashinfo.com/database.htm"

In [4]:
href_url = first_page_pass(url)
good_url = second_page_pass(href_url)
save_csv(good_url,"urls/"+"list_of_list_urls.csv")
k = decompress_list(good_url)
save_csv(k,"urls/"+"list_of_urls.csv")

In [5]:
(len(good_url),len(k))

(105, 5037)

In [9]:
good_url[-20:]

[['https://planecrashinfo.com/2005/2005-1.htm',
  'https://planecrashinfo.com/2005/2005-2.htm',
  'https://planecrashinfo.com/2005/2005-3.htm',
  'https://planecrashinfo.com/2005/2005-4.htm',
  'https://planecrashinfo.com/2005/2005-5.htm',
  'https://planecrashinfo.com/2005/2005-6.htm',
  'https://planecrashinfo.com/2005/2005-7.htm',
  'https://planecrashinfo.com/2005/2005-8.htm',
  'https://planecrashinfo.com/2005/2005-9.htm',
  'https://planecrashinfo.com/2005/2005-10.htm',
  'https://planecrashinfo.com/2005/2005-11.htm',
  'https://planecrashinfo.com/2005/2005-12.htm',
  'https://planecrashinfo.com/2005/2005-13.htm',
  'https://planecrashinfo.com/2005/2005-14.htm',
  'https://planecrashinfo.com/2005/2005-15.htm',
  'https://planecrashinfo.com/2005/2005-16.htm',
  'https://planecrashinfo.com/2005/2005-17.htm',
  'https://planecrashinfo.com/2005/2005-18.htm',
  'https://planecrashinfo.com/2005/2005-19.htm',
  'https://planecrashinfo.com/2005/2005-20.htm',
  'https://planecrashinfo.com

In [7]:
for i in good_url[-20:]:
    time.sleep(3)
    new_list = third_final_pass(i)
    save_csv(new_list,"files/" + i[0].split("/")[-2] + ".csv")